# Scraping Data Table of Toronto Neighborhoods with 'M' Postal Code

Importing the necessary libraries (and ones I might use later on).

In [1]:
import pandas as pd
import numpy as np
import csv
from bs4 import BeautifulSoup
import requests

Scraping and Cleaning Up Table (the full explanations can be found in the 'Clustering and Segmenting' notebook.

In [20]:
# using BeautifulSoup to scrape the data table from wikipedia.
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
source=BeautifulSoup(url, 'lxml')

table=source.find('table', class_='wikitable sortable')

rows=table.findAll('tr')

data=[]
for row in rows:
    data.append([t.text.strip() for t in row.findAll('td')])

# appending the data table into a dataframe

df=pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood']) 
df = df[~df['PostalCode'].isnull()] #to filter out bad rows
df.drop(df[df['Borough']=='Not assigned'].index, axis=0, inplace=True) #deleting 'Not assigned' cells.
df1=df.reset_index()
df2=df1.groupby('PostalCode').agg(lambda x: ','.join(x)) # grouping nrighborhoods with the same postal codes.
df2.loc[df2['Neighborhood']=="Not assigned",'Neighborhood']=df2.loc[df2['Neighborhood']=="Not assigned",'Borough'] # assigning the unnamed neighborhoods with their borough names.
df3=df2.reset_index()
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",") #deleting replicates of borough names from each postal code.

df3.head() # displaying the first 10 rows of the appended and cleansed data of Toronto neighborhoods with 'M' postal codes.

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Reading the Geospatial File for the Latitudes and Longtitues of the Toronto Postal Codes

Obtaining the latitudes and longtitudes of each Toronto 'M' postal codes from the csv file since the geocoder was unresponsive.

In [3]:
df_coor=pd.read_csv('http://cocl.us/Geospatial_data')
df_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Renaming the 'Postal Code' column to 'PostalCode' to prepare the table for merging.

In [4]:
df_coor.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

In [5]:
df_coor.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the data tables 

In [21]:
df_Toronto=pd.merge(df3, df_coor, on='PostalCode', how='inner')
df_Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [7]:
df_Toronto.shape

(103, 5)

There are 103 'M' postal codes in Toronto.

# Visualizing the Toronto Neighborhoods

Importing the necessary libraries for maps/ visualization.

In [8]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.



In [43]:
from folium import plugins
import json
from pandas.io.json import json_normalize

In [11]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geographiclib-1.49   | 32 KB     | ##################################### | 100% 
geopy-1.20.0         | 57 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [12]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [30]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'], df_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [16]:
CLIENT_ID = '03KM0T015RKGIIC5OH30NX5WL1HPALMFUZC4BWRZHEG3QXDK' # your Foursquare ID
CLIENT_SECRET = 'QZTKR2AJBYLPXDEAIE5FIAXE31QEOYG0TRQ34CXZ350HPLVY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 03KM0T015RKGIIC5OH30NX5WL1HPALMFUZC4BWRZHEG3QXDK
CLIENT_SECRET:QZTKR2AJBYLPXDEAIE5FIAXE31QEOYG0TRQ34CXZ350HPLVY


Exploring Downtown Toronto

In [31]:
downtown_data=df_Toronto[df_Toronto['Borough']=='DowntownToronto'].reset_index(drop=True)
downtown_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,DowntownToronto,Rosedale,43.679563,-79.377529
1,M4X,DowntownToronto,"Cabbagetown,St. James Town",43.667967,-79.367675
2,M4Y,DowntownToronto,Church and Wellesley,43.665860,-79.383160
3,M5A,DowntownToronto,"Harbourfront,Regent Park",43.654260,-79.360636
4,M5B,DowntownToronto,"Ryerson,Garden District",43.657162,-79.378937


In [29]:
address = 'Downtown Toronto, Ontario'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


In [33]:
# create map of Manhattan using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='pink',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

In [34]:
downtown_data.loc[0, 'Neighborhood']

'Rosedale'

In [36]:
neighborhood_latitude = downtown_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


### Exploring the top 100 venues in Rosedale in the radius of 800 meters.

In [38]:
radius=800
LIMIT=100

url1='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url1

'https://api.foursquare.com/v2/venues/explore?client_id=03KM0T015RKGIIC5OH30NX5WL1HPALMFUZC4BWRZHEG3QXDK&client_secret=QZTKR2AJBYLPXDEAIE5FIAXE31QEOYG0TRQ34CXZ350HPLVY&ll=43.6795626,-79.37752940000001&v=20180605&radius=800&limit=100'

In [40]:
results = requests.get(url1).json()
# results

In [41]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [44]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Summerhill Market,Grocery Store,43.686265,-79.375458
1,Craigleigh Gardens,Park,43.678099,-79.371586
2,TD Canada Trust,Bank,43.686094,-79.376549
3,TD Canada Trust,Bank,43.672484,-79.377162
4,Mooredale House,Building,43.678631,-79.380091


In [45]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

11 venues were returned by Foursquare.


### Exploring Neighborhoods in Downtown Toronto

In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=800):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results1 = requests.get(url2).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results1])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [47]:
downtown_venues = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )


Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie


In [50]:
print(downtown_venues.shape)
downtown_venues.head()

(1559, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Summerhill Market,43.686265,-79.375458,Grocery Store
1,Rosedale,43.679563,-79.377529,Craigleigh Gardens,43.678099,-79.371586,Park
2,Rosedale,43.679563,-79.377529,TD Canada Trust,43.686094,-79.376549,Bank
3,Rosedale,43.679563,-79.377529,TD Canada Trust,43.672484,-79.377162,Bank
4,Rosedale,43.679563,-79.377529,Mooredale House,43.678631,-79.380091,Building


In [51]:
downtown_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 217 uniques categories.


## Analyzing Each Neighborhood

In [52]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,...,Trail,Train Station,Tunnel,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
downtown_onehot.shape

(1559, 217)

### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [54]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Tunnel,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,...,0.000000,0.00,0.000000,0.00,0.020000,0.00,0.00,0.00,0.00,0.00
1,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.01,0.000000,0.00,0.010000,0.00,0.00,0.00,0.00,0.00
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.00,0.037037,0.037037,0.037037,0.074074,0.111111,0.037037,0.00,...,0.000000,0.00,0.037037,0.00,0.000000,0.00,0.00,0.00,0.00,0.00
3,"Cabbagetown,St. James Town",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00
4,Central Bay Street,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,...,0.000000,0.00,0.000000,0.01,0.020000,0.00,0.00,0.00,0.01,0.00
5,"Chinatown,Grange Park,Kensington Market",0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.00,0.070000,0.00,0.00,0.04,0.01,0.00
6,Christie,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.00,0.016393,0.00,0.00,0.00,0.00,0.00
7,Church and Wellesley,0.010000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,...,0.000000,0.00,0.000000,0.00,0.000000,0.01,0.01,0.01,0.00,0.01
8,"Commerce Court,Victoria Hotel",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,...,0.000000,0.01,0.000000,0.00,0.020000,0.00,0.00,0.00,0.01,0.00
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,...,0.000000,0.01,0.000000,0.00,0.010000,0.00,0.00,0.00,0.01,0.00


In [55]:
downtown_grouped.shape

(18, 217)

#### Top 5 Venues in Each Neighborhood

In [59]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.08
2        Steakhouse  0.04
3             Hotel  0.04
4  Sushi Restaurant  0.03


----Berczy Park----
                 venue  freq
0          Coffee Shop  0.09
1                 Café  0.06
2                Hotel  0.05
3           Restaurant  0.05
4  Japanese Restaurant  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                 venue  freq
0      Airport Service  0.11
1      Harbor / Marina  0.11
2  Rental Car Location  0.07
3          Coffee Shop  0.07
4     Sculpture Garden  0.07


----Cabbagetown,St. James Town----
         venue  freq
0  Coffee Shop  0.09
1   Restaurant  0.06
2  Pizza Place  0.04
3     Pharmacy  0.04
4         Café  0.04


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.08
1  Middle Eastern Restaurant  0.03
2             Ice Cream Shop  0.03


Converting the data above into a pandas dataframe in a descending order

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Top 10 Venues in Each Neighborhood

In [63]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Hotel,Steakhouse,American Restaurant,Bar,Pizza Place,Theater,Sushi Restaurant,Gym
1,Berczy Park,Coffee Shop,Café,Restaurant,Hotel,Beer Bar,Japanese Restaurant,Park,Breakfast Spot,Cocktail Bar,Steakhouse
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Harbor / Marina,Airport Service,Coffee Shop,Rental Car Location,Airport Lounge,Sculpture Garden,Boat or Ferry,Bar,Plane,Music Venue
3,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Café,Pizza Place,Pharmacy,Gastropub,Thai Restaurant,Pub,Beer Store,Park
4,Central Bay Street,Coffee Shop,Japanese Restaurant,Middle Eastern Restaurant,Café,Italian Restaurant,Ice Cream Shop,Park,Burger Joint,Sushi Restaurant,Burrito Place


## Clustering Neighborhood

In [68]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [72]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
downtown_merged['Cluster Labels'] = kmeans.labels_

downtown_merged = downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,DowntownToronto,Rosedale,43.679563,-79.377529,2,Park,Trail,Bank,Playground,Grocery Store,Building,Candy Store,Dessert Shop,Diner,Discount Store
1,M4X,DowntownToronto,"Cabbagetown,St. James Town",43.667967,-79.367675,1,Coffee Shop,Restaurant,Café,Pizza Place,Pharmacy,Gastropub,Thai Restaurant,Pub,Beer Store,Park
2,M4Y,DowntownToronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Café,Gastropub,Gay Bar,Italian Restaurant,Mediterranean Restaurant,Diner,Dance Studio
3,M5A,DowntownToronto,"Harbourfront,Regent Park",43.654260,-79.360636,1,Coffee Shop,Italian Restaurant,Restaurant,Café,Park,Bakery,Pub,Gym / Fitness Center,Theater,Cosmetics Shop
4,M5B,DowntownToronto,"Ryerson,Garden District",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Café,Tea Room,Restaurant,Ramen Restaurant,Gastropub,Cosmetics Shop,Gym


## Visualizing Using a Map

In [83]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining Clusters

### Examining each cluster and determine the discriminating venue categories that distinguish it.

### Cluster 1

In [77]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,DowntownToronto,0,Coffee Shop,Café,Restaurant,Hotel,Bakery,Breakfast Spot,Gastropub,Seafood Restaurant,American Restaurant,Cosmetics Shop
6,DowntownToronto,0,Coffee Shop,Café,Restaurant,Hotel,Beer Bar,Japanese Restaurant,Park,Breakfast Spot,Cocktail Bar,Steakhouse
8,DowntownToronto,0,Coffee Shop,Café,Hotel,Steakhouse,American Restaurant,Bar,Pizza Place,Theater,Sushi Restaurant,Gym
9,DowntownToronto,0,Coffee Shop,Hotel,Café,Aquarium,Italian Restaurant,Restaurant,Brewery,Scenic Lookout,Park,Deli / Bodega
10,DowntownToronto,0,Coffee Shop,Hotel,Café,Bakery,Restaurant,Steakhouse,Gastropub,Bar,Concert Hall,American Restaurant
11,DowntownToronto,0,Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,Gastropub,Concert Hall,Breakfast Spot,Japanese Restaurant,American Restaurant
15,DowntownToronto,0,Coffee Shop,Café,Japanese Restaurant,Restaurant,Beer Bar,Bakery,Gastropub,Seafood Restaurant,Hotel,Cocktail Bar
16,DowntownToronto,0,Coffee Shop,Hotel,Café,Restaurant,Steakhouse,Theater,Gastropub,American Restaurant,Japanese Restaurant,Bakery


Distinguished by coffee shops, cafes, and hotels.

### Cluster 2

In [79]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,DowntownToronto,1,Coffee Shop,Restaurant,Café,Pizza Place,Pharmacy,Gastropub,Thai Restaurant,Pub,Beer Store,Park
2,DowntownToronto,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Café,Gastropub,Gay Bar,Italian Restaurant,Mediterranean Restaurant,Diner,Dance Studio
3,DowntownToronto,1,Coffee Shop,Italian Restaurant,Restaurant,Café,Park,Bakery,Pub,Gym / Fitness Center,Theater,Cosmetics Shop
4,DowntownToronto,1,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Café,Tea Room,Restaurant,Ramen Restaurant,Gastropub,Cosmetics Shop,Gym
7,DowntownToronto,1,Coffee Shop,Japanese Restaurant,Middle Eastern Restaurant,Café,Italian Restaurant,Ice Cream Shop,Park,Burger Joint,Sushi Restaurant,Burrito Place
17,DowntownToronto,1,Korean Restaurant,Coffee Shop,Grocery Store,Café,Indian Restaurant,Diner,Karaoke Bar,Japanese Restaurant,Park,Cocktail Bar


Distinguished by ethnic restaurants: Japanese, Korean, Middle Eastern.

### Cluster 3

In [80]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,DowntownToronto,2,Park,Trail,Bank,Playground,Grocery Store,Building,Candy Store,Dessert Shop,Diner,Discount Store


Distinguished by parks, trails, and banks.

### Cluster 4

In [81]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,DowntownToronto,3,Harbor / Marina,Airport Service,Coffee Shop,Rental Car Location,Airport Lounge,Sculpture Garden,Boat or Ferry,Bar,Plane,Music Venue


Distiguished by a marina (including boats and ferries) and airport services (including car rentals and lounges).

### Cluster 5

In [82]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,DowntownToronto,4,Café,Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Bakery,Bubble Tea Shop,Bookstore,Pizza Place,Pub,Italian Restaurant
13,DowntownToronto,4,Café,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Vietnamese Restaurant,Bakery,Coffee Shop,Dessert Shop,Chinese Restaurant,Caribbean Restaurant


Distinguished by vegetarian/ vegan restaurants, bakeries, and cafes.